<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import pandas as pd
import numpy as np
import random
import sys
import os

titles = 'https://raw.githubusercontent.com/CVanchieri/DS-Unit-4-Sprint-3-Deep-Learning/master/module1-rnn-and-lstm/articles/titles.csv'
sonnets = 'https://raw.githubusercontent.com/CVanchieri/DS-Unit-4-Sprint-3-Deep-Learning/master/module1-rnn-and-lstm/articles/sonnets.csv'
dft = pd.read_csv(titles, header=None)
dfs = pd.read_csv(sonnets, sep = '\n', header=None)

In [0]:
# show the dataframe shape.
print(dft.shape)
# show the dataframe with headers.
dft.head()

(44, 1)


,0
0,THE SONNETS
1,ALL’S WELL THAT ENDS WELL
2,THE TRAGEDY OF ANTONY AND CLEOPATRA
3,AS YOU LIKE IT
4,THE COMEDY OF ERRORS


In [0]:
# show the dataframe shape.
print(dfs.shape)
# show the dataframe with headers.
dfs.head()

(2309, 1)


,0
0,1
1,"From fairest creatures we desire increase,"
2,"That thereby beauty’s rose might never die,"
3,"But as the riper should by time decease,"
4,His tender heir might bear his memory:


In [0]:
dfs = dfs[~dfs[0].str.contains(r'\d')]

In [0]:
# encode data as chars.
text = " ".join(dfs[0].tolist())
chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

In [0]:
# show the length of the chars.
len(chars)

61

In [0]:
# create the squence data.
# The lists are: encoded (chars encoded as integers)
#                sequences (40 chars per entry)
#                next_chars ()
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # each element is 40 chars long.
next_chars = [] # one element for each sequence for prediction.

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences', len(sequences))

sequences 18848


In [0]:
# specify x & y - 3D matrix.

# initialize arrays with False.
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

### base model.

In [0]:
from keras.layers.core import Dropout

# create baseline model.
def create_model():
    # set the sequential model.
    model = Sequential()
    model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #input shape of a singular observ.
    model.add(Dense(len(chars), activation='softmax'))
    # compile the model.
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [0]:
# set the model.
model = create_model()
# fit the model with parameters.
model.fit(x, y, batch_size=20, epochs=2)

Train on 18848 samples
Epoch 1/2
18848/18848 [==============================] - 84s 4ms/sample - loss: 2.8087 - acc: 0.2458
Epoch 2/2
18848/18848 [==============================] - 84s 4ms/sample - loss: 2.3405 - acc: 0.3446


### Batch Size.

In [0]:
# imports.
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


# create the model.
def create_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #input shape of a singular observ.
    model.add(Dense(len(chars), activation='softmax'))

    # compile the model.
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
# set the model with kerasclassifier.
model = KerasClassifier(build_fn=create_model, verbose=0)

In [0]:
# set the parameters for batch size.
param_grid = {'batch_size': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
              }

In [0]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x, y)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.3395580530166626 using {'batch_size': 5}
Means: 0.3395580530166626, Stdev: 0.008818184414304311 with: {'batch_size': 5}
Means: 0.31663904786109925, Stdev: 0.006072138419425553 with: {'batch_size': 10}
Means: 0.3111734390258789, Stdev: 0.007242021112283264 with: {'batch_size': 15}
Means: 0.2996607780456543, Stdev: 0.007364043353493179 with: {'batch_size': 20}
Means: 0.2855479121208191, Stdev: 0.011956522187677952 with: {'batch_size': 25}
Means: 0.27233617305755614, Stdev: 0.008114202905722113 with: {'batch_size': 30}
Means: 0.2667655348777771, Stdev: 0.004188886590738569 with: {'batch_size': 35}
Means: 0.2366294026374817, Stdev: 0.030215467802852638 with: {'batch_size': 40}
Means: 0.19238219559192657, Stdev: 0.009351837686857923 with: {'batch_size': 45}
Means: 0.20431989729404448, Stdev: 0.01648868863783165 with: {'batch_size': 50}


In [0]:
# imports.
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV


# create the model.
def create_model():
    model = Sequential()
    model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #input shape of a singular observ.
    #model.add(Dropout(0.2))
    model.add(Dense(len(chars), activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(len(chars), activation='softmax'))

    #optimizer = Adam(lr=0.01)

    # compile the model.
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [0]:
# set the model with kerasclassifier.
model = KerasClassifier(build_fn=create_model, batch_size=5, epochs=10, verbose=0)

In [0]:
# set the gridsearch.
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(x, y)
# show the results.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

In [0]:
# create sample function.
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
# create test function.
def test(c_length):
    
    start_index = random.randint(0, len(text) - maxlen - 1)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(c_length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = int_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

char_length = 400

In [0]:
# fit the model with parameters.
model.fit(x, y,
          batch_size=128,
          epochs=20)

Train on 18848 samples
Epoch 1/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 1.9365
Epoch 2/20
18848/18848 [==============================] - 23s 1ms/sample - loss: 1.9187
Epoch 3/20
18848/18848 [==============================] - 23s 1ms/sample - loss: 1.9024
Epoch 4/20
18848/18848 [==============================] - 23s 1ms/sample - loss: 1.8865
Epoch 5/20
18848/18848 [==============================] - 23s 1ms/sample - loss: 1.8734
Epoch 6/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 1.8580
Epoch 7/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 1.8450
Epoch 8/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 1.8289
Epoch 9/20
18848/18848 [==============================] - 23s 1ms/sample - loss: 1.8165
Epoch 10/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 1.8037
Epoch 11/20
18848/18848 [==============================] - 24s 1ms/sample - loss: 1.7953
Epoch 1

In [0]:
# use the text function on the char_length, gives test output.
test(char_length)

----- Generating with seed: "eed outbraves his dignity:   For sweetes"
eed outbraves his dignity:   For sweetes goud thou blauty dost my desthed thee blown, And pairt and have be to me the thear enes, And loves not loves whee sheal of thy praie, Mand and love of ance me for mine where with me to me, And arl heaven the preas,   Thou bether with love sighe with por bet, And sond hand I love not head your seake. To ene hat sepe sove more a to me beate, And more sweak come and shand not love eress for, Ere so 


# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN